<a href="https://colab.research.google.com/github/cmd-gunma-univ/DX-highschool/blob/main/wav%E9%8C%B2%E9%9F%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sounddeviceを使った録音

## カメラの接続と確認

USBカメラを接続し、以下のコマンドで接続を確認してください。

In [ ]:
import sounddevice as sd
print(sd.query_devices())          # すべてのデバイス一覧
print("Default device:", sd.default.device)

```
> 1 C270 HD WEBCAM: USB Audio (hw:2,0), ALSA (1 in, 0 out)
```

例えば上記であったおきの説明：  
1: sounddeviceでのデバイス番号  
1 in: モノラル入力のマイク(ステレオの場合は2 in)

## 録音
以下でsounddeviceで5秒間wav録音を行います。  
record.wavというファイル名で保存されます。  
`device_in`の番号が異なる場合は置き換えてください。

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write

fs = 48000         # ← サンプリングレートを48kHz
duration = 5       # ← 5秒間録音する
device_in = 1      # ← sounddeviceでのデバイス番号に置き換える

sd.default.device = (None, device_in)
print("録音開始...")
data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
sd.wait()
write("record.wav", fs, data)
print("✅ 保存しました：record.wav")

## 再生
以下でrecord.wavの再生を行います。

In [ ]:
import os
from IPython.display import Audio

if os.path.exists("record.wav"):
    display(Audio("record.wav"))
else:
    print("record.wav が見つかりません")

## FFT変換と結果のグラフ表示

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

# === WAV 読み込み ===
fs, data = wavfile.read("record.wav")  # 読み込み
print("Sampling rate:", fs)
print("Shape:", data.shape)

# ステレオの場合 → モノラル化（左右平均）
if len(data.shape) == 2:
    data = data.mean(axis=1)

# 正規化（オプション）
data = data / np.max(np.abs(data))

# === FFT ===
N = len(data)  # 総サンプル数
fft_data = np.fft.fft(data)
fft_data = np.abs(fft_data)[:N//2]  # 片側のみ（実信号なので）

# === 周波数軸の作成 ===
freqs = np.fft.fftfreq(N, 1/fs)[:N//2]

# === プロット ===
plt.figure(figsize=(12, 6))
plt.plot(freqs, fft_data)
plt.title("FFT Spectrum of record.wav")
plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude")
plt.grid()
plt.xlim(0, fs/2)  # ナイキスト周波数まで表示
plt.show()
